<a href="https://colab.research.google.com/github/oserikov/transliteration-tur/blob/master/shuffle_tur_wiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### предварительные установки

заканчиваются появлением папки `/content/tur-wikis`, скопированной из гуглпапки DeepNLP Cross-Lingual Morphological Analysis, в которой лежат разные наши вики.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


ссылка на гуглпапку с архивом с вики:
https://drive.google.com/drive/folders/12S8mx9W8O1jKk5g81xMsgjEULyRivbsn?usp=sharing

**добавьте её в свой гуглдиск в корень**, чтобы работало (я там ниже делаю `!cp -R /content/drive/My\ Drive/DeepNLP\ Cross-Lingual\ Morphological\ Analysis/tur-wikis .`) или переправьте код на правильный для вас.

In [0]:
!cp -R /content/drive/My\ Drive/DeepNLP\ Cross-Lingual\ Morphological\ Analysis/tur-wikis .

## настройки

In [0]:
# настраиваемые переменные:
# какие языки добавить в перемешанную выборку
languages = ('bak', 'kaz', 'kir', 'tat', 'tur')  #, 'crh', 'krc') 
# сколько предложений брать из каждого языка
language_rows_num = 2000



# глобальные переменные, использованные в логике 
# их менять не нужно
normalized_file_extension = "txt_norm"
normalized_wikis_dir = 'tur-wikis'

## составляем выборку обучающих предложений

### отфильтровываем "плохие" предложения
`good_line()` будет говорить, "хорошая" ли строка

In [0]:
def good_line(line) -> bool:
    return line.strip() and len(line) > 40

In [5]:
for language in languages:
    !echo "$(date) BEFORE filter out bad sentences for {language}"
    
    src_f = open(f"{normalized_wikis_dir}/{language}.{normalized_file_extension}", 'r', encoding="utf-8")
    tgt_f = open(f"{normalized_wikis_dir}/{language}.no_headers", 'w', encoding="utf-8")
    
    prev_line_was_empty = True
    for line in src_f:
        # filter out all the *bad* lines and article titles
        if not prev_line_was_empty and good_line(line):
            print(line, file=tgt_f, end='')

        prev_line_was_empty = not bool(line.strip())
    
    src_f.close()
    tgt_f.close()

    !echo "$(date) AFTER filter out bad sentences for {language}"

Sat Nov 30 02:46:28 UTC 2019 BEFORE filter out bad sentences for bak
Sat Nov 30 02:46:31 UTC 2019 AFTER filter out bad sentences for bak
Sat Nov 30 02:46:33 UTC 2019 BEFORE filter out bad sentences for kaz
Sat Nov 30 02:46:39 UTC 2019 AFTER filter out bad sentences for kaz
Sat Nov 30 02:46:41 UTC 2019 BEFORE filter out bad sentences for kir
Sat Nov 30 02:46:43 UTC 2019 AFTER filter out bad sentences for kir
Sat Nov 30 02:46:45 UTC 2019 BEFORE filter out bad sentences for tat
Sat Nov 30 02:46:48 UTC 2019 AFTER filter out bad sentences for tat
Sat Nov 30 02:46:51 UTC 2019 BEFORE filter out bad sentences for tur
Sat Nov 30 02:47:03 UTC 2019 AFTER filter out bad sentences for tur


### шафлим полученные хорошие предложения в нужном кол-ве

In [6]:
# выберем language_rows_num случайных хороших предложений из каждого языка
# запишем их в pre_shuffled_data.tsv

!rm pre_shuffled_data.tsv
for language in languages:
    !shuf -n {language_rows_num} \
     {normalized_wikis_dir}/{language}.no_headers \
    | sed -e "s|^|{language}\t|" \
    >> pre_shuffled_data.tsv


# перемешаем pre_shuffled_data.tsv
# запишем результат в shuffled_data.tsv
!shuf pre_shuffled_data.tsv > shuffled_data.tsv

rm: cannot remove 'pre_shuffled_data.tsv': No such file or directory


## посмотрим, что получилось

In [7]:
!head shuffled_data.tsv

kir	Калкы - 4289 киши(2013-ж.). . Аянты - 10,01 км2. Мамлекет ичинде бекитилген расмий коду — "08 1 17 025". Мэр — Лотар Шобер.
tat	Русия дәүләт су реестры мәгълүматы буенча елга Түбән Об су бассейны округында урнашкан, су хуҗалыгы участогы — Об елгасы Иртеш елгасы кушылган урыннан Төньяк Сосва елгасы кушылган урынына кадәр. Кече елга бассейны — Об кушылдыклары бассейны, Иртештән Төньяк Сосьва тамагына кадәр, елга бассейны — (Түбән) Об, Иртеш кушылганга кадәр.
tur	Тахтабаş, Ордu илинин Фатса илçесине баğлы бир махалледир.
kir	Алагушов, Балбай. Кыргыз музыкасы: Энциклопедиялык окуу куралы / Башкы редактору Үсөн Асанов. – Бишкек: Мамлекеттик тил жана энциклопедия борбору, 2004. – 400 бет. -ЫСБН 9967-14-016-X.
bak	Дәүерҙең ниндәй вақыт арауығын биләүенә қарата берҙәм генә фекер юқ. Қайһы бер тарихсылар уның башын XВЫЫ быуат аҙағына, икенселәре XВЫЫЫ быуат уртаһына бәйләй. XВЫЫ быуатта рационализм нигеҙҙәрен үҙенең «Метод тураһында фекер йөрөтөү» тигән хеҙмәтендә Декарт һала (1637). Мәғриф

In [8]:
for language in languages:
    !echo "there are $(grep ^{language} shuffled_data.tsv | wc -l ) samples of language {language}"

there are 2000 samples of language bak
there are 2000 samples of language kaz
there are 2000 samples of language kir
there are 2000 samples of language tat
there are 2000 samples of language tur
